# Imputação de valores faltantes - Experimento

Este é um componente para imputação de valores faltantes usando média, mediana ou mais frequente. Faz uso da implementação do [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

strategy_num = "mean" #@param ["mean", "median", "most_frequent", "constant"] {type:"string", label:"Estratégia de atribuição de valores para colunas numéricas", description:"'mean' é apenas para atributos numéricos, 'median' é apenas para atributos numéricos, 'most_frequent' é para atributos numéricos e categóricos e 'constant' substituirá os valores ausentes por fill_value"}
strategy_cat = "most_frequent" #@param ["most_frequent", "constant"] {type:"string", label:"Estratégia de atribuição de valores para colunas categóricas", description:"'most_frequent' é para atributos numéricos e categóricos e 'constant' substituirá os valores ausentes por fill_value"}

fillvalue_num = 0 #@param {type:"number", label:"Valor de preenchimento para colunas numéricas (utilizado apenas com estratégia 'constant')", description:"Valor a ser preenchido no lugar de valores nulos"}
fillvalue_cat = "" #@param {type:"string", label:"Valor de preenchimento para colunas categóricas (utilizado apenas com estratégia 'constant')", description:"Valor a ser preenchido no lugar de valores nulos"}

## Acesso ao conjunto de dados

Utiliza a função `load_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar conjuntos de dados.
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1)
y = df[target]

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Configuração dos atributos

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

## Treina modelo usando sklearn.impute.SimpleImputer

In [ ]:
import pandas as pd
from platiagro.featuretypes import NUMERICAL
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    make_column_transformer((SimpleImputer(strategy=strategy_num, fill_value=fillvalue_num),
                             numerical_indexes),
                            (SimpleImputer(strategy=strategy_cat, fill_value=fillvalue_cat),
                             non_numerical_indexes),
                            remainder='passthrough'))

# Train model and transform dataset
X = pipeline.fit_transform(X)

features_after_pipeline = \
        np.concatenate((columns[numerical_indexes],
                        columns[non_numerical_indexes]))

# Put data back in a pandas.DataFrame
df = pd.DataFrame(data=X, columns=features_after_pipeline)
df[target] = y

## Salva alterações no conjunto de dados

Utiliza a função `save_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar alterações no conjuntos de dados.

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline,
           columns=columns,
           features_after_pipeline=features_after_pipeline)